In [13]:
begin
    using Pkg
    Pkg.activate("../")
    Pkg.instantiate()
end

include("../modules/DataIO.jl")
include("../modules/Model.jl")

using FASTX,
    .DataIO,
    .Model
    

  Activating project at `~/Desktop/rrm-genomic-extractor`


### Collectiong histogram as datasets

1. Define Hyperparams and data variables

In [5]:
wnwPercent::Float32 = 0.004
variantDirPath::String = "/home/salipe/Desktop/datasets/tutorial_data/kmers"

cachdir::String = "$(pwd())/.project_cache/$wnwPercent"

try
    mkpath(cachdir)
catch e
    @error "create cache direcotry failed" exception = (e, catch_backtrace())
end

"/home/salipe/Desktop/rrm-genomic-extractor/notebooks/.project_cache/0.004"

2. Load Kmers set

In [9]:
variantDirs::Vector{String} = readdir(variantDirPath)
kmerset = Set{String}()

@simd for variant in variantDirs
    variantKmers = DataIO.read_pickle_data("$variantDirPath/$variant/$(variant)_ExclusiveKmers.sav")
    union!(kmerset, Set(variantKmers))
end

kmerset

Set{String} with 4293 elements:
  "TAAAATCAGCACCTT"
  "GGAATTTCAAGAACT"
  "GTCTACTACAGTCAG"
  "GTGGTCTCCCTCAGG"
  "GAACCGACGATGACT"
  "GTTAGTGCTAAATCA"
  "ATGGCTGTGATGCTG"
  "CTTCTAGAGTTCTTG"
  "ATTACATTATAAACC"
  "GTTTTTTTATGTACT"
  "ATAACTTTTAAAGAA"
  "GCTATGCCTTCGAAT"
  "CATCTAGATGGTGAA"
  "CTTGTTTTAAGCTTT"
  "TTACCACAGAAATTT"
  "TAGAGTGTTAGGTTT"
  "GGTTGGGAACTTGTT"
  "GATAATAGGCTTAAA"
  "TACACACAACACATT"
  ⋮ 

3. Loading dataset informatio
    - Load sequences
    - Defining Kmer regex function and Window operation values

In [14]:
input_data = Vector{Tuple{String,Vector{Vector{Float64}}}}(undef, length(variantDirs))

@inbounds for v in eachindex(variantDirs)
    variant::String = variantDirs[v]
    println("Processing $variant")
    cache_path = "$cachdir/$(variant)_outmask.dat"

    cache::Union{Nothing,Vector{Vector{Float64}}} = DataIO.load_cache(cache_path)

    if !isnothing(cache)
        @info "Using cached data from $cache_path"
        outputs[v] = (variant,data)
    else

        sequences = String[]
        for record in open(FASTAReader, "$variantDirPath/$variant/$variant.fasta")
            push!(sequences, sequence(String, record))
        end
        minSeqLength::UInt16 = minimum(map(length, sequences))
        wnwSize::UInt16 = ceil(UInt16, minSeqLength * wnwPercent)

        data = Model.wndwSequencesKmersHistogram(kmerset,wnwSize,sequences)
        DataIO.save_cache(cache_path, data)

        outputs[v] = (variant,data)
        
    end
    println("Finish Processing $variant")
end

Processing Alpha


┌ Info: Cache saved successfully: /home/salipe/Desktop/rrm-genomic-extractor/notebooks/.project_cache/0.004/Alpha_outmask.dat
└ @ Main.DataIO /home/salipe/Desktop/rrm-genomic-extractor/modules/DataIO.jl:124


UndefVarError: UndefVarError: `outputs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.